In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *


In [2]:
spark=SparkSession.builder.appName('desafio-python').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/31 15:46:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [24]:
from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        white-space: pre !important;
    }
    pre { white-space: pre !important; }
</style>
""")

In [33]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.truncate', 30)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 20)

In [4]:
# dataframes_dict = {}
# dataframes_dict['metadata_de_facturas']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/bkpf_vbrk.parquet")
# dataframes_dict['facturas_por_cliente']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/vbpa_kna1.parquet")
# dataframes_dict['materiales']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/vbrp_mara_marm.parquet")
# dataframes_dict['paises']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/T005T.parquet")

########################################################################################################################################

dataframes_dict = {}
dataframes_dict['metadata_de_facturas']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/bkpf_vbrk.parquet")
dataframes_dict['facturas_por_cliente']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/vbpa_kna1.parquet")
dataframes_dict['materiales']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/vbrp_mara_marm.parquet")
dataframes_dict['paises']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/T005T.parquet")

########################################################################################################################################

In [36]:
dataframes_dict['facturas_por_cliente'].select('Pais').distinct()

Pais
FI
NL
MX
CN
SV
null
PT
TW
CL
AU


#### Filtros por Mandante = 400

In [5]:
df_metadata_facturas = dataframes_dict['metadata_de_facturas'].where(col('Mandante') == '400')

In [6]:
df_facturas_por_cliente = dataframes_dict['facturas_por_cliente'].where(col('Mandante') == '400')

In [7]:
df_materiales = dataframes_dict['materiales'].where(col('Mandante') == '400')

In [8]:
df_paises = dataframes_dict['paises'].where(col('Mandante') == '400').where(col('Idioma') == 'S')
# Ocupar sólo "S" Y info mandante 400

#### SELECT DE TABLAS

In [9]:
df_metadata_facturas = df_metadata_facturas.select('Mandante', 'Factura', 'Clase_factura',).where(col('Factura').isNotNull())


In [10]:
df_facturas_por_cliente = df_facturas_por_cliente.select('Mandante','Factura','Posicion_de_Factura','Pais', 'Cliente_Solicitante', 'Cliente_Destinatario_de_Mercancia')

In [11]:
df_materiales = df_materiales.groupBy('Factura', 'Material', 'Unidad_de_medida_Volumen_Destino', 'Pais').agg(sum(col('Volumen')).alias('Volumen'))

In [12]:
df_paises = df_paises.select('Idioma', 'Pais' , 'Pais_descr')

#### Join tablas

In [14]:
def cliente_clase_factura(dataframes_dict):
    df_result = df_facturas_por_cliente.join(df_metadata_facturas, 'Factura', 'left').drop(df_metadata_facturas.Mandante)

    return {'add_cliente_clase_factura': df_result}

def materiales_factura_y_clase_factura(dataframes_dict):
    df_clase_factura = dataframes_dict['add_cliente_clase_factura']
    df_result = df_materiales.join(df_clase_factura, 'Factura', 'left').drop(df_clase_factura.Pais).drop(df_clase_factura.Mandante)

    return {'add_materiales_factura_y_clase_factura': df_result}

def paises_descriptivos(dataframes_dict):
    df_clase_factura = dataframes_dict['add_materiales_factura_y_clase_factura']
    df_paises_desc = df_clase_factura.join(df_paises, 'Pais', 'left')

    return {'add_mat_factura_y_clase_factura': df_paises_desc}



In [27]:
dataframes_dict['add_mat_factura_y_clase_factura'].select('Pais').distinct()

Pais
CL


In [16]:
def gen_srv_desafio_python(dataframes_dict):

    functions_list = [
        cliente_clase_factura,
        materiales_factura_y_clase_factura,
        paises_descriptivos,
        # Otras transformaciones
    ]

    for function in functions_list:
        print(function)
        result_element = function(dataframes_dict)
        dataframes_dict.update(result_element)

    return dataframes_dict['add_mat_factura_y_clase_factura'] # Modificar al último dataframe retornado

In [17]:
test = gen_srv_desafio_python(dataframes_dict)

<function cliente_clase_factura at 0x10c3aa290>
<function materiales_factura_y_clase_factura at 0x107f23f40>
<function paises_descriptivos at 0x107f23a30>


In [19]:
dataframes_dict['materiales'].count()

80708